<a href="https://colab.research.google.com/github/tomonari-masada/course2025-nlp/blob/main/08_evaluating_answer_choice_accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLMを使った多肢選択式質問への応答

* 今回は、とりあえず、生成モデルをちょっと使ってみる。

* 以下の記事を参考にした。
  * https://magazine.sebastianraschka.com/p/llm-evaluation-4-approaches

* ランタイムのタイプをGPUに変更しておこう。

## データセット

* MMLUデータセットから、`high_school_mathematics`の部分を使う。
  * https://huggingface.co/datasets/cais/mmlu

In [ ]:
from datasets import load_dataset

ds = load_dataset("cais/mmlu", "high_school_mathematics")

In [ ]:
ds

* 今回使用するのは`test`スライス
  * 生成モデルをいきなりそのまま使うだけなので。

## プロンプト作成のためのヘルパ関数

In [ ]:
def format_prompt(example):
    return (
        f"{example['question']}\n"
        f"A. {example['choices'][0]}\n"
        f"B. {example['choices'][1]}\n"
        f"C. {example['choices'][2]}\n"
        f"D. {example['choices'][3]}\n"
        "Answer: "  # trailing space encourages a single-letter next token
    )

* プロンプト作成を試してみる。

In [ ]:
ds["test"][0]

* 多肢選択問題になっている。

In [ ]:
print(format_prompt(ds["test"][0]))

## モデルの取得

* 今回はQwenの軽量なLLMにしておく。
  * https://huggingface.co/Qwen/Qwen3-0.6B

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B")

 * 推論するだけなので、evalutionモードに設定する。

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

* トークナイザを試してみる。

In [ ]:
prompt = format_prompt(ds["test"][0])
tok = torch.tensor(tokenizer.encode(prompt), device=device).unsqueeze(0)
tok

* どのようなサブワードに分割されているかを見てみる。

In [ ]:
print(tokenizer.convert_ids_to_tokens(tok[0]))

## 答えの生成

In [ ]:
out = model.generate(tok, max_new_tokens=8, temperature=0.0, do_sample=False)
print(tokenizer.decode(out.squeeze(0)))

* attention maskをちゃんと渡すには以下のようにする。

In [ ]:
input = tokenizer(prompt, return_tensors="pt")
tok = input['input_ids'].to(device)
mask = input['attention_mask'].to(device)

In [ ]:
out = model.generate(tok, attention_mask=mask, max_new_tokens=8, temperature=0.0, do_sample=False)
print(tokenizer.decode(out.squeeze(0)))

In [ ]:
answer = tokenizer.decode(out.squeeze(0)[len(tok[0]):], skip_special_tokens=True)
print(answer)

In [ ]:
pred = None
for letter in answer:
    letter = letter.upper()
    if letter in "ABCD":
        pred = letter
        break
if pred is None:
    pred = "N/A"

ans = ds["test"][0]["answer"]
gold = "ABCD"[ans] if isinstance(ans, int) else str(ans).strip().upper()

print(f"Predicted: {pred}, Correct: {gold}")

* ここまでの処理をヘルパ関数としてまとめる。

In [ ]:
def predict_choice(example):
    prompt = format_prompt(example)

    input = tokenizer(prompt, return_tensors="pt")
    tok = input['input_ids'].to(device)
    mask = input['attention_mask'].to(device)

    out = model.generate(tok, attention_mask=mask, max_new_tokens=8, temperature=0.0, do_sample=False)
    answer = tokenizer.decode(out.squeeze(0)[len(tok[0]):], skip_special_tokens=True)

    pred = None
    for letter in answer:
        letter = letter.upper()
        if letter in "ABCD":
            pred = letter
            break
    if pred is None:
        pred = "N/A"

    ans = example["answer"]
    gold = "ABCD"[ans] if isinstance(ans, int) else str(ans).strip().upper()

    return pred, gold

In [ ]:
predict_choice(ds["test"][0])

## 評価

* どのくらい正答するか、少し様子を見てみる。

In [ ]:
for i in range(10):
    pred, gold = predict_choice(ds["test"][i])
    print(f"Q{i+1}: Predicted: {pred}, Correct: {gold}")

* `test`スライス全体で正解率を求める。

In [ ]:
import time

total = 0
correct = 0
start = time.time()
for i in range(len(ds["test"])):
    pred, gold = predict_choice(ds["test"][i])
    total += 1
    if pred == gold:
        correct += 1
    if (i + 1) % 10 == 0:
        end = time.time()
        print(f"Processed {i+1}/{len(ds['test'])} in {end - start:.1f} sec")
        start = end
        print(f"  Current accuracy: {correct}/{total} = {correct/total:.3%}")
end = time.time()
print(f"Accuracy: {correct}/{total} = {correct/total:.3%}")